In [1]:
import pandas as pd

pretest_data_path = "./data/test.xlsx"
pretest_data = pd.read_excel(pretest_data_path)

pretest_data

,conversations,Blame,error
0,A1: 亲亲，这里是客服奥克兰，很高兴为您服务~----💥6.18开门红，疯抢中~~🕒【半价...,NaN,NaN
1,A1: 亲亲，您好，在的呢。这里是售前客服长城，很荣幸为您服务！​​​\nQ: 我家是旧水槽...,NaN,NaN
2,A1: 亲亲，您好，在的呢。这里是售前客服长城，很荣幸为您服务！​​​\nQ: ？\nA0:...,NaN,NaN
3,A1: 亲亲，这里是客服智利，很高兴为您服务~----💥6.18开门红，疯抢中~~🕒【半价】...,NaN,NaN
4,A1: 亲亲，您好，在的呢。这里是售前客服智利，很荣幸为您服务！​​​\nQ: 有没有独立的...,NaN,NaN
...,...,...,...
195,A1: 亲亲，您好，在的呢。这里是售前客服荷兰，很荣幸为您服务！​​​\nA1: 您好，这边...,NaN,NaN
196,A1: 亲亲，这里是客服利比亚，很高兴为您服务~----💥6.18狂欢节，疯抢中~~活动时间...,NaN,NaN
197,A1: 非常抱歉给您带来困扰，您的问题已经收到了，巴库正在加速赶来，请亲亲稍等一下哈​​​\...,NaN,NaN
198,A1: 亲亲，您好，在的呢。这里是售前客服贝宁，很荣幸为您服务！​​​\nQ: 发货时间\n...,NaN,NaN


In [2]:
from dotenv import load_dotenv
import os


load_dotenv()

True

## OpenRouter Integration

In [3]:
# from langchain.chat_models import init_chat_model
# openai_api_key = os.getenv("OPENROUTER_API_KEY")
# openai_api_base = "https://openrouter.ai/api/v1"

# llm_model = "qwen/qwen3-235b-a22b:free"
# llm_model = "qwen/qwen3-32b:free"
# model_name = "qwen"
# llm_model = "deepseek/deepseek-v3-base:free"
# model_name = "deepseek"
# llm_model = "microsoft/phi-4-reasoning:free"
# model_name = 'phi4reason'
# llm_model = "nousresearch/deephermes-3-mistral-24b-preview:free"
# model_name = "deephermes"

# model = init_chat_model(
#     model=llm_model,
#     model_provider="openai",
#     openai_api_base=openai_api_base,
#     openai_api_key=openai_api_key,
# )

## Azure OpenAI Integration

In [4]:
# from langchain_openai import AzureChatOpenAI

# model = AzureChatOpenAI(
#     azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
#     azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
#     openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
#     api_key=os.environ["AZURE_API_KEY"],
# )

# model_name = "gpt"

## Gemini Integration

In [5]:
from langchain.chat_models import init_chat_model

model = init_chat_model(
    # "gemini-2.0-flash",
    "gemini-2.5-flash-preview-04-17",
    model_provider="google_genai",
    api_key=os.environ["GEMINI_API_KEY"],
)

model_name = 'gemini'

In [6]:
import time

def invoke_with_retry(model, messages, max_retries, retry_delay):
    for attempt in range(max_retries):
        try:
            return model.invoke(messages)
        except Exception as e:
            print(
                f"Attempt {attempt + 1} failed: {e}. Retrying in {retry_delay} seconds..."
            )
            time.sleep(retry_delay)
    raise Exception("Max retries exceeded.")

In [7]:
from model import blame_prompt, error_prompt
from langchain_core.messages import HumanMessage, SystemMessage

max_retries = 20
retry_delay = 5  # seconds

conversations = pretest_data["conversations"]
results = []

for conv in conversations:
    blame_messages = [
        SystemMessage(blame_prompt),
        HumanMessage(conv),
    ]
    error_messages = [
        SystemMessage(error_prompt),
        HumanMessage(conv),
    ]
    
    blame_res = invoke_with_retry(model, blame_messages, max_retries, retry_delay)
    error_res = invoke_with_retry(model, error_messages, max_retries, retry_delay)
    
    blame_flag = 1 if blame_res.content == 'True' else 0
    error_flag = 1 if error_res.content == 'True' else 0
    print('----------------------------------')
    # print("Original Conversation:\n", conv, sep='')
    print("Blame:", blame_flag)
    print("Error:", error_flag)

    results.append({
        "Conversation": conv,
        "Blame": blame_flag,
        "Error": error_flag
    })

df = pd.DataFrame(results)
df

----------------------------------
Blame: 0
Error: 0
----------------------------------
Blame: 1
Error: 1
----------------------------------
Blame: 1
Error: 1
----------------------------------
Blame: 0
Error: 0
----------------------------------
Blame: 0
Error: 0
----------------------------------
Blame: 0
Error: 0
----------------------------------
Blame: 0
Error: 1
----------------------------------
Blame: 0
Error: 0
----------------------------------
Blame: 0
Error: 0
----------------------------------
Blame: 0
Error: 1
----------------------------------
Blame: 0
Error: 1
----------------------------------
Blame: 0
Error: 1
----------------------------------
Blame: 0
Error: 0
----------------------------------
Blame: 0
Error: 0
----------------------------------
Blame: 0
Error: 0
----------------------------------
Blame: 0
Error: 0
----------------------------------
Blame: 0
Error: 0
----------------------------------
Blame: 0
Error: 0
----------------------------------
Blame: 0
Er

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 10
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 58
}
].


----------------------------------
Blame: 0
Error: 0
----------------------------------
Blame: 0
Error: 0
----------------------------------
Blame: 0
Error: 0
----------------------------------
Blame: 0
Error: 1
----------------------------------
Blame: 0
Error: 1
----------------------------------
Blame: 0
Error: 1
----------------------------------
Blame: 0
Error: 1
----------------------------------
Blame: 0
Error: 1
----------------------------------
Blame: 1
Error: 1
----------------------------------
Blame: 1
Error: 1
----------------------------------
Blame: 0
Error: 1
----------------------------------
Blame: 0
Error: 0
----------------------------------
Blame: 0
Error: 0
----------------------------------
Blame: 0
Error: 1
----------------------------------
Blame: 0
Error: 1
----------------------------------
Blame: 0
Error: 1
----------------------------------
Blame: 0
Error: 0
----------------------------------
Blame: 0
Error: 0
----------------------------------
Blame: 0
Er

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 10
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 2
}
].


Attempt 1 failed: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 10
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 59
}
]. Retrying in 5 seconds...
----------------------------------
Blame: 0
Error: 1
----------------------------------
Blame: 1
Error: 1
----------------------------------
Blame: 1
Error: 1
----------------------------------
Blame: 0
Error: 0
----------------------------------
Blame: 0
Error: 0
-------------------------------

...

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 10
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 3
}
].


Attempt 1 failed: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 10
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 1
}
]. Retrying in 5 seconds...
----------------------------------
Blame: 0
Error: 1
----------------------------------
Blame: 1
Error: 1
----------------------------------
Blame: 0
Error: 0
----------------------------------
Blame: 1
Error: 1
----------------------------------
Blame: 0
Error: 1
--------------------------------

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 10
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 4
}
].


Attempt 1 failed: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 10
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 1
}
]. Retrying in 5 seconds...
----------------------------------
Blame: 0
Error: 1
----------------------------------
Blame: 0
Error: 1
----------------------------------
Blame: 1
Error: 1
----------------------------------
Blame: 0
Error: 0
----------------------------------
Blame: 0
Error: 0
--------------------------------

,Conversation,Blame,Error
0,A1: 亲亲，这里是客服奥克兰，很高兴为您服务~----💥6.18开门红，疯抢中~~🕒【半价...,0,0
1,A1: 亲亲，您好，在的呢。这里是售前客服长城，很荣幸为您服务！​​​\nQ: 我家是旧水槽...,1,1
2,A1: 亲亲，您好，在的呢。这里是售前客服长城，很荣幸为您服务！​​​\nQ: ？\nA0:...,1,1
3,A1: 亲亲，这里是客服智利，很高兴为您服务~----💥6.18开门红，疯抢中~~🕒【半价】...,0,0
4,A1: 亲亲，您好，在的呢。这里是售前客服智利，很荣幸为您服务！​​​\nQ: 有没有独立的...,0,0
...,...,...,...
195,A1: 亲亲，您好，在的呢。这里是售前客服荷兰，很荣幸为您服务！​​​\nA1: 您好，这边...,0,1
196,A1: 亲亲，这里是客服利比亚，很高兴为您服务~----💥6.18狂欢节，疯抢中~~活动时间...,1,1
197,A1: 非常抱歉给您带来困扰，您的问题已经收到了，巴库正在加速赶来，请亲亲稍等一下哈​​​\...,0,0
198,A1: 亲亲，您好，在的呢。这里是售前客服贝宁，很荣幸为您服务！​​​\nQ: 发货时间\n...,0,0


In [8]:
import datetime
current_time = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M")
df.to_csv(f'./out/llm_label_test_result_{current_time}_{model_name}.csv')